<a href="https://colab.research.google.com/github/ahmedabdurahim/TheGarageSale/blob/main/TheGarageSaleDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is the database manipulation and fyp algorithm part of the TheGarageSale bot written in a separate file which is then imported as TheGarageSaleDB in the main file

In [ ]:
# Import code

from pymongo import MongoClient
import urllib
import re
import datetime
cluster = MongoClient("--- MongoDB api link ---")

InsertUserId() method adds the userid given to it through the parameter to the database. But first it checks if it exists, if it does already exist then it does not add the user id. If it doesn't exist then it adds the user id to the database

In [ ]:
def InsertUserId(userId):
    db = cluster['thegaragesale']
    collection = db['User_Id']
    if collection.find_one({'user id': userId}) != None:
        pass
    else:
        post = {"user id": userId, "balance": "0", "state": "start", "messageID": "", "ApparelBrowse": 0, "AutomobileBrowse": 0, "ElectronicsBrowse": 0, "FoodAndDrinksBrowse": 0, "FreeItemsBrowse": 0, "HardwareToolsBrowse": 0, "ListItemsCount": 0}
        collection.insert_one(post)

analyticsUpdate() this method updates how the analytics values for the user_id on the database to be later used in fyp algorithms

In [ ]:
def analyticsUpdate(userId, value, valueUpdated):
    db = cluster['thegaragesale']
    collection = db['User_Id']
    filter = {'user id': userId}
    newvalues = {"$set": {value: valueUpdated}}
    collection.update_one(filter, newvalues)

exists() this method checks for duplicate listings, it takes the description of the listing and the table it is going to be posted to as another parameter and then returns true if the listing already exists and false if it doesn't

In [ ]:
def exists(description,collection):
    db = cluster['thegaragesale']
    c = re.split("\s",collection)[1]
    if(c=="Apparel"):
        collection = db['Apparel and clothes']
    elif(c=="Electronics"):
        collection = db['Electronics']
    elif (c == "Automobile"):
        collection = db['Automobile & parts']
    elif (c == "Automobile"):
        collection = db['Automobile & parts']
    elif (c == "Hardware"):
        collection = db['Hardware/tools ']
    elif (c == "Food"):
        collection = db['Food and Drinks']
    elif (c == "Free"):
        collection = db['Free items']
    if collection.find_one({'Description': description}) != None:
        return True

    return False


userUpdate() methond is used to update fields from the user_id's document in the database.

In [ ]:
def userUpdate(userId,param,value):
    db = cluster['thegaragesale']
    collection = db['User_Id']
    filter = {'user id': userId}
    if(param=="price"):
        newvalues = {"$set": {'price': value}}
    elif(param=="skip"):
        newvalues = {"$set": {'skip': value}}
    elif(param == "tableName"):
        newvalues = {"$set": {'tableName': value}}
    elif (param == "photo_id"):
        newvalues = {"$set": {'photo_id': value}}
    elif (param == "l_id"):
        newvalues = {"$set": {'l_id': value}}
    elif (param == "boost"):
        newvalues = {"$set": {'boost': value}}
    elif (param == "onProfile"):
        newvalues = {"$set": {'onProfile': value}}
    elif (param == "onProfileDescription"):
        newvalues = {"$set": {'onProfileDescription': value}}
    elif (param == "purchase"):
        newvalues = {"$set": {'purchase': value}}
    else:
        newvalues = {"$set": {'description': value}}
    collection.update_one(filter, newvalues)

userGet() gets values from the document in the database through specifications given from the parameters

In [ ]:
def userGet(userId,param):
    db = cluster['thegaragesale']
    collection = db['User_Id']
    Id = collection.find_one({'user id': userId})
    if Id == None:
        pass
    else:
        return Id[str(param)]

stateUpdate() updates the 'state' field in the user_id document in the database. It helps to keep track of where the user is and what they are doing.

In [ ]:
def stateUpdate(userId, stateName):
    db = cluster['thegaragesale']
    collection = db['User_Id']
    filter = {'user id': userId}
    newvalues = {"$set": {'state': stateName}}
    collection.update_one(filter, newvalues)

messageID() when called it saves the message id of the give message id to the field 'messageID' on the database in the user_id table. this will prove usefull to clearing historical messages to make the user experience more clean 

In [ ]:
def messageID(userId, messageID):
    db = cluster['thegaragesale']
    collection = db['User_Id']
    # msgID = ""

    col = collection.find({'user id': userId})
    for cl in col:
        msgID = cl['messageID']

    filter = {'user id': userId}
    newvalues = {"$set": {'messageID': str(msgID) + " " + str(messageID)}}
    collection.update_one(filter, newvalues)

stateGet() Gets the current place the user is navigating in the bot

In [ ]:
def stateGet(userId):
    db = cluster["thegaragesale"]
    collection = db['User_Id']
    Id = collection.find_one({'user id': userId})

    if Id == None:
        pass
    else:
        return Id['state']

RetrieveUserId() this method has no distnict use, was used for testing puropses. It prints true if the user id is found in the database and false otherwise

In [ ]:
def RetrieveUserId(userId):
    db = cluster["thegaragesale"]
    collection = db['User_Id']
    Id = collection.find_one({'user id': userId})

    if Id == None:
        pass
    elif Id['user id'] == userId:
        print("true")
    else:
        print("false")

The GetBalance() method will return the balance for that specific user

In [ ]:
def GetBalance(userId):
    db = cluster["thegaragesale"]
    collection = db['User_Id']
    Id = collection.find_one({'user id': userId})

    if Id == None:
        pass
    else:
        return Id['balance']


UpdateBalance() method will update the balance of that specific user, to a new balance given in the parameter

In [ ]:
def UpdateBalance(userId, newBalance):
    db = cluster["thegaragesale"]
    collection = db['User_Id']
    filter = {'user id': userId}
    newvalues = {"$set": {'balance': newBalance}}
    collection.update_one(filter, newvalues)

PostListing() method is the one that commits new listings to the database. It is used in the main file with the post() method

In [ ]:
def PostListing(category, userId, price, description,photoId,LId,Boost):
    db = cluster['thegaragesale']
    collection = db[category]
    post = {"Description": description,
            "user_id": userId,
            "price": price,
            "photo_Id": photoId,
            "l_id":LId,
            "boost": Boost
            }

    collection.insert_one(post)

allPostsForUserId() method returns the array of all posts description from the user.

In [ ]:
def allPostsForUserId(userId):
    Posts = []
    list = ["Apparel and clothes", "Automobile & parts", "Electronics", "Food and Drinks", "Free items", "Gift cards",
            "Hardware/tools"]
    db = cluster['thegaragesale']
    for i in list:
        collection = db[i]
        X = collection.find({"user_id": userId})
        for x in X:
            Posts.append(x['Description'])
    return Posts

GetDocumentID() gets the document id of a listing that matches the parameters given to it. used for the callback query for the 'purchase this' button in the main file

In [ ]:
def GetDocumentID(TableName, description, price):
    documentID = None
    db = cluster["thegaragesale"]
    collection = db[str(TableName)]
    X = collection.find({})
    for x in X:
        if(x['Description'] == str(description) and x['price'] == price):
            #print('db file works great')
            documentID = x['_id']
    return documentID

deleteSpecificDocument() deletes the document from the database after taking the document id

In [ ]:
def deleteSpecificDocument(id):
    db=cluster["thegaragesale"]
    list = ["Apparel and clothes", "Automobile & parts", "Electronics", "Food and Drinks", "Free items", "Gift cards",
            "Hardware/tools"]
    for i in list:
        col=db[i]
        filter={'_id': id}
        col.delete_one(filter)

getDescription() gets the description of a listing, it takes the parameters 'table' which is table name and 'docId' which is the document id, or the unique id for a listing

In [ ]:
def getDescription(table,docId):
    db = cluster["thegaragesale"]
    collection = db[f'{table}']
    col = collection.find({'_id': docId})
    for cl in col:
        return str(cl['Description'])

get() method fetches the photo id, price and the user id of a listing that was posted. It then returns the whole thing as a string and we split it in the main file with regular expression to make use of it

In [ ]:
def get(table,docId):
    db = cluster["thegaragesale"]
    collection = db[f'{table}']
    col = collection.find({'_id': docId})
    for cl in col:
       return  (str(cl['l_id'])+  " "+ str(cl['price']) + " " +str(cl['user_id']))

GetTopBoost() returns the value of the top boosted listing in the bot. It takes the table name to search for as a parameter

In [ ]:
def GetTopBoost(category):
    PostData = {'boost': 0, 'description': '', 'price': 0, 'photoId': '', 'userId': 0, 'l_id': ''}
    db = cluster["thegaragesale"]
    collection = db[category]
    X = collection.find({})
    for x in X:
        if x['boost'] > PostData['boost']:
            PostData['boost'] = x['boost']
            PostData['description'] = x['Description']
            PostData['price'] = x['price']
            PostData['photoId'] = x['photo_Id']
            PostData['userId'] = x['user_id']
            PostData['l_id'] = x['l_id']

    tgs = db['the garage sale']
    filter = {'Category': str(category)}
    newvalues = {"$set": {'Description': PostData['description'],
                         'user_id': PostData['userId'],
                         'price': PostData['price'],
                         'photo_Id': PostData['photoId'],
                         'l_id': PostData['l_id'],
                         'boost': PostData['boost']
                        }
                }
    tgs.update_one(filter, newvalues)

    return PostData['boost']

The GetUserTendency() method gets the intrests of that specific user to be later used in the FYP functionality. it returns the value as a string and finds the percentage of visitation of a specific category to the total. we then multiply the decimal value by 10 and round it to 0 digits/ no decimals. Finally we concatenate this data to a string and return it. The idea of this code is that for example if the results for 'ApparelBrowse' was 6 then each page on the FYP page would contain 6 Apparel related posts. Each page contains 10 listings. This part is still an idea and not yet implemented fullscale.

In [ ]:

def GetUserTendency(userId):
    db = cluster["thegaragesale"]
    collection = db['User_Id']
    col = collection.find_one({'user id': userId})

    Tendency = {'ApparelBrowse': col['ApparelBrowse'], 
                'AutomobileBrowse': col['AutomobileBrowse'],
                'ElectronicsBrowse': col['ElectronicsBrowse'],
                'FoodAndDrinksBrowse': col['FoodAndDrinksBrowse'],
                'FreeItemsBrowse': col['FreeItemsBrowse'],
                'HardwareToolsBrowse': col['HardwareToolsBrowse']
               }
    added = sum(Tendency.values())
    # print(added)
    return(str(int(round(Tendency['ApparelBrowse']/added * 10, 0))) 
           + " " + str(int(round(Tendency['AutomobileBrowse']/added * 10, 0))) 
           + " " + str(int(round(Tendency['ElectronicsBrowse']/added * 10, 0))) 
           + " " + str(int(round(Tendency['FoodAndDrinksBrowse']/added * 10, 0)))
           + " " + str(int(round(Tendency['FreeItemsBrowse']/added * 10, 0)))
           + " " + str(int(round(Tendency['HardwareToolsBrowse']/added * 10, 0))))